In [1]:
# A generic genetic algorithm
import random
import numpy as np
import matplotlib.pyplot as plt
from client import get_errors, get_overfit_vector, submit
from io import StringIO
from config import SECRET_KEY
from utils import POPULATION_SIZE, SPLIT_POINT, FEATURE_VECTOR_SIZE, MUTATION_COEFF
from helpers import mkdir_p, get_train_validation_errors, get_rank_and_probabilities, fitness, select_parents, cross_over, mutate, plot_graph, initialize_params, select_the_best, return_elite, list_to_array

from errno import EEXIST
from os import makedirs, path

In [17]:
a = [ 2.69969038e+00, -8.98587638e+00, -5.91113586e+00, -6.63182532e+00,
  -1.75214813e-10, -1.83669770e-15,  8.52944060e-16,  2.29423303e-05,
  -2.04721003e-06, -1.59792834e-08,  9.98214034e-10,]
x = get_errors(SECRET_KEY, list(a))
y = submit(SECRET_KEY, list(a))
print(y)
print(x)

successfully submitted
[89625927779.16754, 322595357543.82635]


In [18]:
a = [ 0.00000000e+00,  6.27937040e+00,  3.19484293e+00, -3.72370557e+00,
  -1.75214813e-10, -1.83669770e-15, -7.58848480e+00,  2.29423303e-05,
  -2.04721003e-06, -1.59792834e-08,  9.98214034e-10,]
#x = get_errors(SECRET_KEY, list(a))
y = submit(SECRET_KEY, list(a))
print(y)
print(x)

successfully submitted
[89625927779.16754, 322595357543.82635]


In [19]:
a = [-7.83878833e+00,
  -8.27116100e-01,
  3.19484293e+00, 
  -4.37903432e+00, 
  -1.75214813e-10, 
  -1.83669770e-15, 
  -7.58848480e+00, 
  2.29423303e-05, 
  -2.04721003e-06, 
  -1.59792834e-08,
  9.98214034e-10]
#x = get_errors(SECRET_KEY, list(a))
y = submit(SECRET_KEY, list(a))
print(y)
print(x)
# 1, 2, 3, 4
# 1, 2

authentication/usage quota error
[89625927779.16754, 322595357543.82635]


In [9]:
params = [[-7.83878833e+00, -7.83878833e+00, -7.83878833e+00, -7.83878833e+00,
  -7.83878833e+00, -7.83878833e+00, -7.83878833e+00,  3.57243504e+00,
  -7.83878833e+00, -7.83878833e+00],
 [-8.27116100e-01,  8.30381203e+00,  8.30381203e+00, -8.27116100e-01,
   8.30381203e+00,  8.30381203e+00, -8.27116100e-01,  8.30381203e+00,
   8.30381203e+00,  8.30381203e+00],
 [ 3.19484293e+00,  3.19484293e+00,  3.19484293e+00,  3.19484293e+00,
   3.19484293e+00,  3.19484293e+00,  3.19484293e+00,  3.19484293e+00,
   3.19484293e+00,  3.19484293e+00],
 [-4.37903432e+00, -3.72370557e+00, -3.72370557e+00, -4.37903432e+00,
  -3.72370557e+00, -3.72370557e+00, -4.37903432e+00, -3.72370557e+00,
  -3.72370557e+00, -3.72370557e+00],
 [-1.75214813e-10, -1.75214813e-10, -1.75214813e-10, -1.75214813e-10,
   9.28021666e+00, -9.82741237e+00, -1.75214813e-10, -1.75214813e-10,
  -1.75214813e-10, -1.75214813e-10],
 [-1.83669770e-15, -1.83669770e-15, -1.83669770e-15, -1.83669770e-15,
  -1.83669770e-15, -1.83669770e-15, -1.83669770e-15, -1.83669770e-15,
  -1.83669770e-15, -1.83669770e-15],
 [-7.58848480e+00, -7.58848480e+00, -7.58848480e+00, -7.58848480e+00,
  -7.58848480e+00, -7.58848480e+00,  8.63071069e+00, -7.58848480e+00,
  -7.58848480e+00, -7.58848480e+00],
 [ 2.29423303e-05,  2.29423303e-05,  2.29423303e-05,  2.29423303e-05,
   2.29423303e-05,  2.29423303e-05, -3.88193090e+00,  2.29423303e-05,
   2.29423303e-05,  2.29423303e-05],
 [-2.04721003e-06, -2.04721003e-06, -2.04721003e-06, -2.04721003e-06,
  -2.04721003e-06, -2.04721003e-06, -2.04721003e-06, -2.87756078e+00,
   5.86159126e+00, -2.04721003e-06],
 [-1.59792834e-08, -1.59792834e-08, -1.59792834e-08, -1.59792834e-08,
  -1.59792834e-08, -1.59792834e-08, -1.59792834e-08, -1.59792834e-08,
  -1.59792834e-08, -8.05504898e+00],
 [ 9.98214034e-10,  9.98214034e-10,  9.98214034e-10,  9.98214034e-10,
   9.98214034e-10,  9.98214034e-10,  9.98214034e-10,  9.98214034e-10,
   9.98214034e-10,  9.98214034e-10]]
print(len(params[0]), len(params))
params = np.array(params)
print(params.shape)

10 11
(11, 10)


In [12]:
init_params = np.zeros(shape=(11, 10), dtype=float)
np.copyto(init_params, params)

In [16]:
init_overfit_params, init_params = initialize_params()
NO_OF_GENERATIONS = 10
generation_errors = np.zeros(shape=(NO_OF_GENERATIONS, 2), dtype=float)
params = np.zeros(shape=(11, POPULATION_SIZE), dtype=float)
np.copyto(params, init_params)
print("Initial params:\n")
print(init_params)
target_directory = "Results/3-03/run4"
mkdir_p(target_directory)
f = open("{}/params.txt".format(target_directory), "w")
#f.write("Run 3 of 3rd March is continued, we started with the final parameters of previous run i.e. run3 and simulated x more generations where x =\n" + str(NO_OF_GENERATIONS) + "\n")
f.write("Starting all over again with new root population\n")
f.write("No of generations: " + str(NO_OF_GENERATIONS) + "\n")
f.write("Population size: " + str(POPULATION_SIZE) + "\n")
f.write("Mutation coefficient: " + str(MUTATION_COEFF) + "\n")
f.write("Split point: " + str(SPLIT_POINT) + "\n")
f.write("Two offspring per pair\n")

for i in range(NO_OF_GENERATIONS):
    train_validation_errors = None
    fitness_score_array = None
    rank_array = None
    # fitness
    train_validation_errors = get_train_validation_errors(SECRET_KEY, POPULATION_SIZE, params)
    fitness_score_array = fitness(np.array(train_validation_errors))
    rank_array, selection_prob = get_rank_and_probabilities(fitness_score_array)
    elite_parents = return_elite(rank_array, POPULATION_SIZE, 2)
    final_parents = select_the_best(rank_array, POPULATION_SIZE, int(POPULATION_SIZE*0.6))
    parents = []
    parent_fitness = np.zeros(shape = (len(final_parents), ), dtype = float)
    k = 0
    temp_errors = list_to_array(train_validation_errors)
    elite_params = []
    for j in elite_parents:
        elite_params.append(params[:, j])
        #parent_fitness[k] = fitness_score_array[j]
        #k += 1
    for j in final_parents:
        parents.append(params[:, j])
        parent_fitness[k] = fitness_score_array[j]
        k += 1
    print(parent_fitness)
    # Parents are ready
    elite_params = list_to_array(elite_params)
    parents = list_to_array(parents)
    parent_rank_array, parent_selection_prob = get_rank_and_probabilities(parent_fitness)
    
    f.write("Population:\n")
    f.write(str(params))
    f.write("\nTrain validation errors\n")
    f.write(str(train_validation_errors))
    f.write('\n')
    f.write("Selected parents:\n")
    f.write(str(parents))
    for j in range(POPULATION_SIZE):
        if rank_array[j]==1:
            generation_errors[i, :] += train_validation_errors[j]
    print("Rank array")
    print(rank_array)
    print("All errors")
    print(train_validation_errors)
    print("Best offspring Gen errors")
    print(generation_errors)
    # crossover and mutation
    temp_count = 2
    offspring_params = np.zeros(shape=(11, POPULATION_SIZE), dtype=float)
    offspring_params[:, 0] = elite_params[0, :]
    offspring_params[:, 1] = elite_params[1, :]
    while temp_count < POPULATION_SIZE:
        k, l = select_parents(parent_selection_prob)
        #print(k, l)
        print("What's the matter with you")
        offspring1, offspring2 = cross_over(parents[k, :], parents[l, :], SPLIT_POINT)
        #print(offspring)
        offspring1 = mutate(offspring1)
        offspring2 = mutate(offspring2)
        #print(offspring)
        offspring_params[:, temp_count] += offspring1[:]
        offspring_params[:, temp_count+1] += offspring2[:]
        #print(offspring_params)
        temp_count += 2
    #print("Errors")
    #print(generation_errors)
    #print("params")
    #print(offspring_params)
    np.copyto(params, offspring_params)
f.write('Best offspring gen errors:\n')
f.write(str(generation_errors))
f.close()
plot_graph(NO_OF_GENERATIONS, generation_errors, target_directory)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 6.27937040e+00 -1.45799022e-12 -1.45799022e-12 -9.75835723e+00
  -1.45799022e-12 -1.45799022e-12 -8.98587638e+00 -1.45799022e-12
  -1.45799022e-12 -1.45799022e-12]
 [-2.28980078e-13 -2.28980078e-13 -2.17347038e+00 -2.28980078e-13
  -2.28980078e-13 -2.28980078e-13 -4.25719383e+00 -2.28980078e-13
  -2.28980078e-13  9.72667176e+00]
 [ 4.62010753e-11  4.62010753e-11  4.62010753e-11  4.62010753e-11
   4.62010753e-11  4.62010753e-11  4.62010753e-11 -4.17136006e+00
   4.62010753e-11  4.62010753e-11]
 [-1.75214813e-10 -1.75214813e-10 -1.75214813e-10 -1.75214813e-10
  -1.75214813e-10 -1.75214813e-10 -1.75214813e-10 -1.75214813e-10
  -1.75214813e-10 -1.75214813e-10]
 [-1.83669770e-15 -1.83669770e-15 -1.83669770e-15 -1.83669770e-15
  -1.83669770e-15 -1.83669770e-15 -1.83669770e-15 -1.83669770e-15
  -1.83669770e-15  1.31367627e+00

[-3.50627976e+11 -3.47008662e+11 -3.45724478e+11 -3.50627976e+11
 -3.50628435e+11 -3.08769360e+11]
Rank array
[ 4  3 10  9  2  8  5  6  1  7]
All errors
[[14670596104.554321, 350627975890.82245], [14191135877.565577, 347008661905.23987], [2.462864642611329e+43, 2.506227071561615e+43], [3.605389865620206e+34, 6.820929914742405e+34], [14419703891.673935, 345724478303.13367], [1.7312006391677782e+24, 3.3832226641289714e+24], [14670596104.554321, 350627975890.82245], [14670724499.215328, 350628435351.69574], [36643866303.08591, 308769359566.24585], [2.2348193430892317e+23, 4.3674296287219675e+23]]
Best offspring Gen errors
[[1.33094625e+10 3.66141689e+11]
 [1.46706797e+10 3.50628275e+11]
 [1.46706797e+10 3.50628275e+11]
 [1.46705961e+10 3.50627976e+11]
 [1.41911359e+10 3.47008662e+11]
 [3.66438663e+10 3.08769360e+11]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]]
What's the matter with you
What's the matt

KeyboardInterrupt: 

In [ ]:
def trash3():
    init_overfit_params, init_params = initialize_params()
    NO_OF_GENERATIONS = 3
    generation_errors = np.zeros(shape=(NO_OF_GENERATIONS, 2), dtype=float)
    params = np.zeros(shape=(11, POPULATION_SIZE), dtype=float)
    np.copyto(params, init_params)

    train_validation_errors = get_train_validation_errors(SECRET_KEY, POPULATION_SIZE, params)
    fitness_score_array = fitness(np.array(train_validation_errors))
    rank_array, selection_prob = get_rank_and_probabilities(fitness_score_array)
    elite_parents = return_elite(rank_array, POPULATION_SIZE, 2)
    final_parents = select_the_best(rank_array, POPULATION_SIZE, int(POPULATION_SIZE*0.9))

In [ ]:
def trash2():
    train_validation_errors = get_train_validation_errors(SECRET_KEY, POPULATION_SIZE, params)
    fitness_score_array = fitness(np.array(train_validation_errors))
    rank_array, selection_prob = get_rank_and_probabilities(fitness_score_array)
    elite_parents = return_elite(rank_array, POPULATION_SIZE, 2)
    final_parents = select_the_best(rank_array, POPULATION_SIZE, int(POPULATION_SIZE*0.4))
    parents = []
    print(elite_parents)
    print(final_parents)
    for i in elite_parents:
        parents.append(params[:, i])
    for i in final_parents:
        parents.append(params[:, i])
    print(parents)
    parents = np.array(parents)
    print(parents)

In [ ]:
def trash():
    parent_fitness = np.zeros(shape = (len(elite_parents) + len(final_parents), 2), dtype = float)
    k = 0
    temp_errors = list_to_array(train_validation_errors)
    for i in elite_parents:
    #parents.append(params[:, i])
        parent_fitness[k, :] = temp_errors[i, :]
        k += 1
    for i in final_parents:
        #parents.append(params[:, i])
        parent_fitness[k, :] += temp_errors[i, :]
        k += 1
    print(parent_fitness)

In [ ]:
#elite_parents.size
elite_params[1, :]

In [ ]:
train_validation_errors[:][1]

In [ ]:
#get_overfit_vector(SECRET_KEY)
parent1[0:SPLIT_POINT]

In [ ]:
#get_overfit_vector(SECRET_KEY)
parent_fitness.size
parent_fitness = np.zeros(shape = (len(elite_parents) + len(final_parents)), dtype = float)
parent_fitness.size

In [ ]:
#get_errors()
k, l = select_parents(parent_selection_prob)
offspring1, offspring2 = cross_over(parents[k, :], parents[l, :], SPLIT_POINT)
print(offspring1)

In [ ]:
def dumb():
    init_overfit_params, init_params = initialize_params()
    NO_OF_GENERATIONS = 3
    generation_errors = np.zeros(shape=(NO_OF_GENERATIONS, 2), dtype=float)
    params = np.zeros(shape=(11, POPULATION_SIZE), dtype=float)
    np.copyto(params, init_params)
    print("Initial params:\n")
    print(init_params)
    target_directory = "Results/3-01/run9"
    mkdir_p(target_directory)
    f = open("{}/params.txt".format(target_directory), "w")
    #f.write("Run 1 of 27th feb is continued, we started with the final parameters of previous run i.e. run4 and simulated x more generations where x =\n" + str(NO_OF_GENERATIONS) + "\n")
    f.write("Starting all over again with new root population\n")
    f.write("No of generations: " + str(NO_OF_GENERATIONS) + "\n")
    f.write("Population size: " + str(POPULATION_SIZE) + "\n")
    f.write("Mutation coefficient: " + str(MUTATION_COEFF) + "\n")
    f.write("Split point: " + str(SPLIT_POINT) + "\n")
    f.write("One offspring per pair\n")

    for i in range(NO_OF_GENERATIONS):
        train_validation_errors = None
        fitness_score_array = None
        rank_array = None
        # fitness
        train_validation_errors = get_train_validation_errors(SECRET_KEY, POPULATION_SIZE, params)
        fitness_score_array = fitness(np.array(train_validation_errors))
        rank_array, selection_prob = get_rank_and_probabilities(fitness_score_array)
        f.write("Population:\n")
        f.write(str(params))
        f.write("\nTrain validation errors\n")
        f.write(str(train_validation_errors))
        f.write('\n')
        for j in range(POPULATION_SIZE):
            if rank_array[j]==1:
                generation_errors[i, :] += train_validation_errors[j]
        print("Rank array")
        print(rank_array)
        print("All errors")
        print(train_validation_errors)
        print("Best offspring Gen errors")
        print(generation_errors)
        # crossover and mutation
        temp_count = 0
        offspring_params = np.zeros(shape=(11, POPULATION_SIZE), dtype=float)
        while temp_count < POPULATION_SIZE:
            k, l = select_parents(selection_prob)
            #print(k, l)
            offspring1, offspring2 = cross_over(params[:, k], params[:, l], SPLIT_POINT)
            #print(offspring)
            offspring1 = mutate(offspring1)
            offspring2 = mutate(offspring2)
            #print(offspring)
            offspring_params[:, temp_count] += offspring1[:]
            offspring_params[:, temp_count+1] += offspring2[:]
            #print(offspring_params)
            temp_count += 2
        #print("Errors")
        #print(generation_errors)
        #print("params")
        #print(offspring_params)
        np.copyto(params, offspring_params)

    f.close()
    plot_graph(NO_OF_GENERATIONS, generation_errors, target_directory)